In [ ]:
import numpy as np
import pickle
import scipy.ndimage
import matplotlib.pyplot as plt
import math

In [ ]:
tracks = pickle.load(open("tracks.pkl"))

tracksToKeep=['track2']
kept = []
    

for track in sorted(tracks[1]):
    trackID = track[0]
    trackDict = track[1]
    trackDat = []
    for t in sorted(trackDict):
        trackDat.append([t] + list(trackDict[t][1]))
    trackDat = np.array(trackDat)
    if trackID in tracksToKeep:
        kept.append(trackDat)
    #plt.plot(trackDat[:,0],256-trackDat[:,2])
    #plt.show()
    
for trackID in sorted(tracks[0]):
    track = tracks[0][trackID]
    trackDict = track
    trackDat = []
    for t in sorted(trackDict):
        trackDat.append([t] + list(trackDict[t][1]))
    trackDat = np.array(trackDat)
    if trackID in tracksToKeep:
        kept.append(trackDat)
    #plt.plot(trackDat[:,0],256-trackDat[:,2])
    #plt.show()
track = np.vstack(kept)
track = track
plt.plot(track[:,0],256-track[:,2])
plt.show()


In [ ]:
def jumps(vals):
    
    return [t for t in range(1,len(vals)) if  np.abs(vals[t]-vals[t-1]) > 1]

def samey_intervals(vals, t_window=5):
    last = vals[0]
    start = -1
    accum = 0
    intervals = []
    
    for t, v in enumerate(vals):
        
        if v != last:
            accum += 1
            
        if last == v:
            accum = 0
            
        if accum > 1:
            last = v
            
            if (t-accum)-start > t_window:
                intervals.append(start+1)
                intervals.append(t-accum)
                intervals.append(t-accum+1)

            start = t-accum
            accum = 0

    return intervals


def zero_crossings(vals):    

    sign_intervals = []
    last_sign_change = 0
    vsigns = np.sign(vals)
    last_sign = vsigns[0]    
    zeros = np.zeros(vsigns.shape)
    zeros[vsigns == 0] = 1
    accum = 0

    for t in range(len(zeros)):        
        if zeros[t] == 1:
            accum += 1
        else:
            accum = 0
        zeros[t] = accum
    last = 0

    for t,v in enumerate(zeros):
        if v == 1:
            sign_intervals.append(t)
        elif v == 0 and last > 5:
            sign_intervals.append(t-1)
        last = v

    return sorted(set(sign_intervals))

In [ ]:
axis = 2

velocities = track[1:,axis]-track[:-1,axis] 

same_window = 6

switch_points = set(zero_crossings(velocities))

samey_points = set(samey_intervals(velocities,t_window=same_window))

velocity_times = sorted(switch_points | samey_points | set(jumps(velocities)))

print len(velocity_times)

plt.plot(track[:,2])
plt.plot(np.array(velocity_times),track[np.array(velocity_times,dtype='int'),2],'rx')


import fceulib
inputVec = fceulib.readInputs('movie.fm2')

def button_change_times(button_masks):
    last_mask = 0
    mask_times = []
    # Note: for a temporary optimization we could drop certain buttons?
    for t, b in enumerate(button_masks):
        if t < track[0,0]:
            continue
        if t > track[-1,0]:
            break
        if b != last_mask:
            mask_times.append(t-track[0,0])
            last_mask = b
    return mask_times

button_times = button_change_times(inputVec)
all_times = sorted(set(button_times) | set(velocity_times))
plt.plot(np.array(button_times),track[np.array(button_times),axis],'rx')
plt.show()


In [ ]:
'''def make_inputs(start,end,input_type):
    end = all_times[end]
    start = all_times[start]
    interval = end-start
    data = np.zeros((interval,1))
    
    vels = np.copy(velocities[start:end])
    if input_type in ['cN','accN']:
        data = np.hstack((data,np.ones((interval,1))))
    if input_type in ['acc0','accP','accN']:
        dat = np.ones((interval,1))
        for ii in range(interval):
            dat[ii,0] = ii
            
        data = np.hstack((data,dat))
    if input_type in ['accP','cP']:
        begin = velocities[start-1]
        vels -= begin
    return data, vels
'''
def make_inputs(start,end,input_type):
    end = all_times[end]
    start = all_times[start]
    interval = end-start
    
    if input_type in ['c0']:
        data = np.zeros((interval,1))
    elif input_type in ['acc0']:
        dat = np.ones((interval,1))
        for ii in range(interval):
            dat[ii,0] = ii
        data = dat
    else:
        data = np.ones((interval,1))
    
    vels = np.copy(velocities[start:end])
    if input_type in ['accP','accN']:
        dat = np.ones((interval,1))
        for ii in range(interval):
            dat[ii,0] = ii
            
        data = np.hstack((data,dat))
    return data, vels

In [ ]:
%%time
import statsmodels.api as sm

all_times = [t for t in all_times if (t >= 0 and t < len(velocities))]
likes = [None]*len(all_times)

print "Points:",len(all_times)
print all_times
plt.plot(velocities[:all_times[-1]])
plt.plot(np.array(all_times),velocities[np.array(all_times,dtype='int')],'rx')
plt.show()

# Takahashi Meijin constant, 60 frames / 16 inputs ~= 4 frames per input.
# But note that in general transitions may happen more frequently due to collisions, etc.
min_interval = 1 
max_interval = 1*60
likes = [None]*len(all_times)
for i in range(0,len(all_times)):
    print i
    likes[i] = [None]*len(all_times)
    t0 = all_times[i]
    min_likelihood = float('inf')
    for j in [j for j in range(i+1, len(all_times)) if all_times[j]-all_times[i] > min_interval and  all_times[j]-all_times[i] < max_interval]:
        models = {}
        if all_times[j]-all_times[i] > 1:
            for model in ['c0','cN','accN']:
                X,Y = make_inputs(i,j,model)
                ols = sm.OLS(Y,X,hasconst=False)
                results = ols.fit()
                models[model] = results
        likes[i][j] = models

#print ''

In [ ]:
from scipy import stats

inp = []
def penalty(length,params):
    penalty_type = 'BIC' # 'BIC'
    if penalty_type == 'MDL':
        return 2*(1.0 + 0.5*np.log(length))*(params)
    if penalty_type == 'BIC':
        return 2*np.log(length)*(params)

In [ ]:
%%time
modes = [(0,None)]*(len(all_times))
        
model_type_params = {'c0':1,
                    'cN':1.5,
                    'acc0':2,
                    'accN':2}
str_out = ''
for j in range(1, len(all_times)):
    least = float("inf")
    least_template = None
    for i in range(0, j):
        data = likes[i][j]
        if data:
            for model_type,model in data.items():
                crit = model.bic
                if crit < 0:
                    #print 'A',(all_times[j]-all_times[i]),np.log(all_times[j]-all_times[i])*len(model.params)
                    crit = -1*(all_times[j]-all_times[i]) + np.log(all_times[j]-all_times[i])*len(model.params)
                crit2 = model.model.loglike(model.params)
                m_prev = modes[i][0]
                inp,_ = make_inputs(i,j,model_type)
                cost = penalty(all_times[-1]-all_times[0],model_type_params[model_type])#penalty(all_times[-1]-all_times[0],model.params)
                here = crit + m_prev + cost
                str_out += '{}, {}, {}, {}, {}, {}, {}\n'.format(all_times[i], all_times[j], crit2,crit, cost,here,least)
                if here < least:
                    
                    least = here
                    # prev_i,this_j,t0,t1,name,summary,criterion
                    least_template = (i,j,(model_type,model))
    
                   
    modes[j] = (least, least_template)
    
with open('temp.txt','wb') as outfile:
    outfile.write(str_out)


In [ ]:

def get_path(modes):
    mj = len(modes)-1
    path = [modes[mj]]
    while mj > 0:
        mj = modes[mj][1][0]
        path.append(modes[mj])
    return list(reversed(path))[1:]

path = get_path(modes)
for ii,p in enumerate(path):
    print ii,p[0],'\n',all_times[p[1][0]],all_times[p[1][1]],p[1][2][1].params,'\n'

In [ ]:
cross = {}
selves = {}
for ii,mode in enumerate(path):
    model1 = mode[1][2][1]
    model1_type = mode[1][2][0]
    model1_params = model1.params
    model1 = model1.model
    x1,y1 =  make_inputs(mode[1][0], mode[1][1], model1_type)
    ols = sm.OLS(y1,x1,hasconst=False)
    results = ols.fit()
    crit = -ols.loglike(results.params)#results.bic #-model2.loglike(0.5*(model1_params+model2_params))
    if crit == float('-inf'):
        crit = -1*len(y1) #+ len(model1_params)*np.log(len(y1))
    selves[ii] = crit
    for jj,mode2 in enumerate(path):
        model2 = mode2[1][2][1]
        model2_type = mode2[1][2][0]
        model2_params = model2.params
        model2 = model2.model
        if model1_type == model2_type:
            x2,y2 =  make_inputs(mode2[1][0], mode2[1][1], model2_type)
            xs = np.vstack((x1,x2))
            
            ys = np.vstack((y1.reshape(-1,1),y2.reshape(-1,1)))
            
            ols = sm.OLS(ys,xs,hasconst=False)
            results = ols.fit()
            crit = -ols.loglike(results.params) #results.bic #-model2.loglike(0.5*(model1_params+model2_params))
            if crit == float('-inf'):
                crit = -1*len(ys) #+ len(model1_params)*np.log(len(ys))
        else:
            crit = float('inf')
        cross[(ii,jj)] = crit

In [ ]:
"""UnionFind.py

Union-find data structure. Based on Josiah Carlson's code,
http://aspn.activestate.com/ASPN/Cookbook/Python/Recipe/215912
with significant additional changes by D. Eppstein.
"""

class UnionFind:
    """Union-find data structure.

    Each unionFind instance X maintains a family of disjoint sets of
    hashable objects, supporting the following two methods:

    - X[item] returns a name for the set containing the given item.
      Each set is named by an arbitrarily-chosen one of its members; as
      long as the set remains unchanged it will keep the same name. If
      the item is not yet part of a set in X, a new singleton set is
      created for it.

    - X.union(item1, item2, ...) merges the sets containing each item
      into a single larger set.  If any item is not yet part of a set
      in X, it is added to X as one of the members of the merged set.
    """

    def __init__(self):
        """Create a new empty union-find structure."""
        self.weights = {}
        self.parents = {}

    def __getitem__(self, object):
        """Find and return the name of the set containing the object."""

        # check for previously unknown object
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root
        
    def __iter__(self):
        """Iterate through all items ever found or unioned by this structure."""
        return iter(self.parents)

    def union(self, *objects):
        """Find the sets containing the objects and merge them all."""
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r],r) for r in roots])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest

In [ ]:

unions = UnionFind()
for d in sorted(cross):
    
    t1 = all_times[path[d[0]][1][1]]-all_times[path[d[0]][1][0]]
    t2 = all_times[path[d[1]][1][1]]-all_times[path[d[1]][1][0]]
    
    good = True
    for t in [cross[d], cross[(d[0],d[0])], cross[(d[1],d[1])], cross[(d[1],d[0])]]:
        if t == float('inf'):
            good = False
    if not good:
        continue
    complexityWeight = penalty(t1,model_type_params[path[d[0]][1][2][0]])
    complexityWeight += penalty(t2,model_type_params[path[d[1]][1][2][0]])
    
    joinedWeight =  penalty(t1+t1,model_type_params[path[d[0]][1][2][0]])
    #print path[d[0]][1][2][1].params,path[d[1]][1][2][1].params
    separate = selves[d[0]] + selves[d[1]] + complexityWeight
    joined = cross[d]  + joinedWeight
    print d, selves[d[0]],selves[d[1]],separate,cross[d],joined
    #print complexityWeight,joinedWeight
    if (joined <= separate) or path[d[0]][1][2][0] == 'c0':
        unions.union(d[0],d[1])
        
merged = {}
for u in unions:
    if unions[u] not in merged:
        merged[unions[u]] = set()
    merged[unions[u]].add(u)
print len(merged)
for m in merged:
    print m, merged[m]

In [ ]:
all_times2 = all_times[0:100]
plt.plot(velocities[all_times2[0]:all_times2[-1]])
plt.plot(np.array(all_times2)-all_times2[0],velocities[np.array(all_times2,dtype='int')],'rx')

m2i = {m:i for i,m in enumerate(merged)}
print m2i
colors = ['r','g','b','c','m','y','k','#ff8800','#0088ff','#ff0088','#88ff00','#00ff88','#8800ff']
merged_params = {}
for m in merged:
    models = sorted(merged[m])
    sub = models[0]
    x,y = make_inputs(path[sub][1][0],path[sub][1][1],path[sub][1][2][0])
    interval = all_times[path[sub][1][1]]-all_times[path[sub][1][0]]
    params =  path[sub][1][2][1].params*interval
    
    total = interval
    for sub in models[1:]:
        interval = all_times[path[sub][1][1]]-all_times[path[sub][1][0]]
        
        x_,y_ = make_inputs(path[sub][1][0],path[sub][1][1],path[sub][1][2][0])
        x = np.vstack((x,x_))
        y = np.vstack((y.reshape(-1,1),y_.reshape(-1,1)))
        params +=  path[sub][1][2][1].params*interval
        total += interval
        
    
    merged_params[m] = sm.OLS(y,x,hasconst=False).fit().params#params/float(total)

for u in sorted(unions):
    t0 = all_times[path[u][1][0]]-all_times2[0]
    t1 = all_times[path[u][1][1]]-all_times2[0]
    u_ = m2i[unions[u]]
    if t0 < all_times2[-1]-all_times2[0] and t1 >= 0:
        print t0, t1, u_, path[u][1][2][0] ,merged_params[unions[u]]
        plt.plot([t0,t1],[u_+5,u_+5])#,colors[u])
        
        
plt.show()


In [ ]:




pickle.dump((modes, path, merged, unions, track, all_times,merged_params), open("modes.pkl", 'wb'))